# Converting, modifying and saving models

COBREXA.jl can export JSON and MATLAB-style model formats, which can be
useful when exchanging the model data with other software.

For a test, let's download and open a SBML model:

In [1]:
using COBREXA

!isfile("e_coli_core.xml") &&
    download("http://bigg.ucsd.edu/static/models/e_coli_core.xml", "e_coli_core.xml");

sbml_model = load_model("e_coli_core.xml")

Metabolic model of type SBMLModel
sparse([41, 23, 51, 67, 61, 65, 1, 7, 19, 28  …  72, 3, 8, 33, 57, 66, 31, 45, 46, 57], [1, 2, 2, 2, 3, 3, 4, 4, 4, 4  …  93, 94, 94, 94, 94, 94, 95, 95, 95, 95], [-1.0, -1.0, -1.0, 1.0, -1.0, 1.0, 1.0, -1.0, 1.0, -1.0  …  1.0, -1.0, 1.0, -1.0, 1.0, 1.0, 1.0, -1.0, -1.0, 1.0], 72, 95)
Number of reactions: 95
Number of metabolites: 72


You can save the model as `.json` or `.mat` file using the
`save_model` function:

In [2]:
save_model(sbml_model, "converted_e_coli.json")
save_model(sbml_model, "converted_e_coli.mat")

## Using serialization for quick loading and saving

If you are saving the models only for future processing in Julia environment,
it is often wasteful to encode the models to external formats and decode them
back. Instead, you can use the "native" Julia data format, accessible with
package `Serialization`.

This way, you can use `serialize` to save any model format (even the
complicated `StandardModel`, which does not have a "native" file format
representation):

In [3]:
using Serialization

sm = convert(StandardModel, sbml_model)

open(f -> serialize(f, sm), "myModel.stdmodel", "w")

The models can then be loaded back using `deserialize`:

In [4]:
sm2 = deserialize("myModel.stdmodel")
issetequal(metabolites(sm), metabolites(sm2))

true

This form of loading operation is usually pretty quick:

In [5]:
t = @elapsed deserialize("myModel.stdmodel")
@info "Deserialization took $t seconds"

┌ Info: Deserialization took 0.002112422 seconds
└ @ Main.##277 string:2


Notably, large and complicated models with thousands of reactions and
annotations can take tens of seconds to decode properly. Serialization allows
you to minimize this overhead, and scales well to tens of millions of
reactions.

## Converting and saving a modified model

To modify the models easily, it is useful to convert them to a format that
simplifies this modification. You may use e.g. `CoreModel` that
exposes the usual matrix-and-vectors structure of models as used in MATLAB
COBRA implementations, and `StandardModel` that contains structures,
lists and dictionaries of model contents, as typical in Python COBRA
implementations. The object-oriented nature of `StandardModel` is
better for making small modifications that utilize known identifiers of model
contents.

Conversion of any model to `StandardModel` can be performed using the
standard Julia `convert`:

In [6]:
sm = convert(StandardModel, sbml_model)

Metabolic model of type StandardModel
sparse([41, 23, 51, 67, 61, 65, 1, 7, 19, 28  …  72, 3, 8, 33, 57, 66, 31, 45, 46, 57], [1, 2, 2, 2, 3, 3, 4, 4, 4, 4  …  93, 94, 94, 94, 94, 94, 95, 95, 95, 95], [-1.0, -1.0, -1.0, 1.0, -1.0, 1.0, 1.0, -1.0, 1.0, -1.0  …  1.0, -1.0, 1.0, -1.0, 1.0, 1.0, 1.0, -1.0, -1.0, 1.0], 72, 95)
Number of reactions: 95
Number of metabolites: 72


The conversion can be also achieved right away when loading the model, using
an extra parameter of `load_model`:

In [7]:
sm = load_model(StandardModel, "e_coli_core.json")

Metabolic model of type StandardModel
sparse([9, 51, 55, 64, 65, 34, 44, 59, 66, 64  …  20, 22, 23, 25, 16, 17, 34, 44, 57, 59], [1, 1, 1, 1, 1, 2, 2, 2, 2, 3  …  93, 93, 94, 94, 95, 95, 95, 95, 95, 95], [1.0, 1.0, -1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, 1.0  …  1.0, -1.0, 1.0, -1.0, -1.0, 1.0, -1.0, 1.0, 1.0, -1.0], 72, 95)
Number of reactions: 95
Number of metabolites: 72


As an example, we change an upper bound on one of the reactions:

In [8]:
sm.reactions["PFK"].ub = 10.0

10.0

After [possibly applying more modifications](04_standardmodel.md), you can again save the
modified model in a desirable exchange format:

In [9]:
save_model(sm, "modified_e_coli.json")
save_model(sm, "modified_e_coli.mat")

More information about `StandardModel` internals is available [in a
separate example](04_standardmodel.md).

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*